In [3]:
import pandas as pd
import numpy as np
from fastai import *
from fastai.text import *

In this notebook I am to solve the Spanish Airline Tweets Sentiment Analysis challenge from Kaggle.

https://www.kaggle.com/c/spanish-arilines-tweets-sentiment-analysis/overview/description

I have trained a model language on the spanish wikipedia that I will employ for the Kaggle task. 

In [12]:
# import kaggle .csv file
df = pd.read_csv('./tweets_public.csv')
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,neutral,False,0,0,Trabajar en #Ryanair como #TMA: https://t.co/r...,NaN,Fri Nov 03 12:05:12 +0000 2017,926419989107798016,NaN,Madrid
1,neutral,True,0,0,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,NaN,Sun Nov 26 18:40:28 +0000 2017,934854385577943041,NaN,Mexico City
2,negative,False,0,0,Sabiais que @Iberia te trata muy bien en santi...,NaN,Mon Dec 25 15:40:45 +0000 2017,945318406441635840,NaN,Madrid
3,negative,False,0,0,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,NaN,Mon Nov 06 14:18:35 +0000 2017,927540721296568320,NaN,Pacific Time (US & Canada)
4,positive,True,0,0,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,NaN,Mon Jan 01 23:00:57 +0000 2018,947965901332197376,NaN,Buenos Aires


In [19]:
df.isna().sum()

airline_sentiment    0
text                 0
dtype: int64

In [13]:
df = df.loc[:, ['airline_sentiment', 'text']]
df.airline_sentiment.replace(['negative', 'neutral', 'positive'], [0, 1, 2], inplace=True)

In [14]:
df

,airline_sentiment,text
0,1,Trabajar en #Ryanair como #TMA: https://t.co/r...
1,1,@Iberia @FIONAFERRER Cuando gusten en Cancún s...
2,0,Sabiais que @Iberia te trata muy bien en santi...
3,0,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...
4,2,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...
...,...,...
7862,0,@Iberia @iberiaexpress especialistas en dejart...
7863,1,"Con @Iberia, mi destino a un solo click. ¡Dese..."
7864,2,@Iberia Muy bien. Muchas gracias
7865,0,Es que volar con Ryanair es peor que irte a ch...


Dataset ready, now we are going to mount the drive account to get to our:

* data model (spanish wikipedia)
* pretrained weights of the language model
* vocabulary of the model



In [15]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
train_df = df

Creating data bunch object in fast ai, this process 

In [23]:
!nvidia-smi

Thu Aug 20 21:05:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
bs = 192

In [45]:
data_lm = (TextList.from_df(df, cols='text',)
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [61]:
data_lm.vocab.itos[440:450] # We are also capturing emojis

['italia',
 '🛫',
 'dm',
 'san',
 'tax',
 'alemania',
 'llega',
 '@aireuropa',
 'tampoco',
 '😍']

In [65]:
# Loading weights of the spanish wiki
!mkdir eswiki
!mkdir eswiki/models

mkdir: cannot create directory ‘eswiki/models’: File exists


In [68]:
!cp /content/drive/My\ Drive/training_eswiki/models/es_wt.pth /content/eswiki/models
!cp /content/drive/My\ Drive/training_eswiki/models/es_wt_vocab.pkl /content/eswiki/models


In [73]:
data_lm.path = '/content/eswiki'

In [74]:
lm_fns = ['es_wt', 'es_wt_vocab']

In [75]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [79]:
learn_lm.data.single_ds

LabelList (786 items)
x: LMTextList
xxbos xxmaj línea xxmaj directa , xxmaj ryanair y la larga lista de empresas que xxunk con los 902 https : / / t.co / xxunk vía xxunk,xxbos xxmaj con @iberia , mi destino a un solo click . ¡ xxmaj deseadme suerte ! # holaargentina . xxmaj hola 79 destinos europeos al mejor precio https : / / t.co / xxunk,xxbos @iberia sabemos si se cancela este vuelo ? ? xxmaj gracias ! https : / / t.co / xxunk,xxbos @iberia ¿ es posible cambiar la fecha de mi billete a través de su página web ?,xxbos @iberia xxmaj buenas tardes , es posible que el servicio xxup ib xxmaj plus xxmaj oro te tenga más de 15 minutos en espera para una xxunk … https : / / t.co / xxunk
y: LMLabelList
,,,,
Path: .

In [80]:
lr = 1e-3
lr *= bs/48

In [81]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,6.297725,3.699950,0.342374,00:06
1,4.819430,3.366233,0.401004,00:05


In [82]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.463773,3.157094,0.427455,00:08
1,3.329311,2.977403,0.453571,00:08
2,3.180807,2.840838,0.465327,00:08
3,3.037075,2.834662,0.470573,00:08
4,2.902515,2.723954,0.484821,00:09
5,2.787369,2.707082,0.485491,00:09
6,2.682709,2.691767,0.488021,00:09
7,2.602543,2.694229,0.488132,00:09


In [105]:
learn_lm.predict('mi vuelo no sale', 4)

'mi vuelo no sale solo sin otra forma'

In [84]:
learn_lm.save('es_fine_tuned')
learn_lm.save_encoder('es_fine_tuned_enc')

In [87]:
path = data_lm.path

In [ ]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols='text')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='airline_sentiment')
    .databunch(bs=bs, num_workers=1))

In [101]:
data_clas

TextClasDataBunch;

Train: LabelList (7081 items)
x: TextList
xxbos xxmaj trabajar en # xxmaj ryanair como # xxup tma : https : / / t.co / xxunk # empleo,xxbos @iberia xxup @fionaferrer xxmaj cuando xxunk en xxmaj cancún se viaja y xxunk de manera sin igual,xxbos xxmaj xxunk que @iberia te trata muy bien en santiago de xxunk cambia el asiento , te manda a volar en el xxunk xxunk , e … https : / / t.co / xxunk,xxbos xxup nunca xxup nunca xxup nunca xxunk el café de xxmaj ryanair . 
  xxmaj bueno , nada que xxunk a bordo .,xxbos @cris_tortu @dakar @iberia @mitsubishi_es @bfgoodricheu @burgostur @astintlogistics @uremovil @karbium xxmaj muchos éxito … https : / / t.co / xxunk
y: CategoryList
1,1,0,0,2
Path: /content/eswiki;

Valid: LabelList (786 items)
x: TextList
xxbos xxmaj línea xxmaj directa , xxmaj ryanair y la larga lista de empresas que xxunk con los 902 https : / / t.co / xxunk vía xxunk,xxbos xxmaj con @iberia , mi destino a un solo click . ¡ xxmaj deseadme suerte ! # holaargent

In [90]:
data_clas.save('es_textlist_class')

In [91]:
data_clas = load_data(path, 'es_textlist_class', bs=bs, num_workers=1)

In [93]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy]).to_fp16()
learn_c.load_encoder('es_fine_tuned_enc')
learn_c.freeze()

In [94]:
lr=2e-2
lr *= bs/48

In [95]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,1.016114,0.965652,0.512723,00:01
1,0.950363,0.917150,0.557252,00:01


In [96]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.949734,0.977706,0.520356,00:01
1,0.939973,0.919892,0.550891,00:01


In [97]:
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.939646,0.962590,0.496183,00:01
1,0.922235,0.922146,0.548346,00:01


In [98]:
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.907420,0.945816,0.531807,00:01
1,0.900948,0.916779,0.553435,00:01


In [99]:
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.874687,0.915366,0.555980,00:01


In [100]:
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.871139,0.921229,0.559796,00:01


In [106]:
learn_c.predict('hola que pasa')

/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  idx_min = (t != self.pad_idx).nonzero().min()


(Category tensor(1), tensor(1), tensor([0.3546, 0.4473, 0.1981]))

In [108]:
learn_c.data.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '/',
 ':',
 'de',
 'https',
 't.co',
 '.',
 '@iberia',
 ',',
 'en',
 'a',
 '…',
 'que',
 'y',
 'el',
 'la',
 '#',
 'iberia',
 'con',
 'no',
 '!',
 'ryanair',
 'un',
 'por',
 'los',
 'me',
 '?',
 'vuelo',
 'para',
 'una',
 'mi',
 'es',
 'al',
 'del',
 'se',
 'lo',
 'las',
 '...',
 'hola',
 'más',
 '¡',
 '-',
 'si',
 'madrid',
 'vuelos',
 '\n ',
 'solo',
 'gracias',
 '@ryanair',
 'mejor',
 'pero',
 'desde',
 'su',
 'destino',
 'suerte',
 'precio',
 'destinos',
 'ya',
 'como',
 'europeos',
 'click',
 '79',
 'deseadme',
 'españa',
 '¿',
 'te',
 '2',
 'he',
 '4',
 '(',
 'o',
 ')',
 'así',
 'avión',
 'años',
 'huelga',
 'sus',
 '"',
 'sin',
 'cuando',
 'fue',
 'muy',
 'yo',
 'pilotos',
 'maleta',
 'vía',
 'hace',
 'holacolombia',
 'hay',
 'nos',
 'ha',
 'holaargentina',
 'qué',
 '5',
 'hacer',
 'euros',
 'spanair',
 'días',
 'ni',
 'tengo',
 'porque',
 'q',
 'web',
 '@ryanair_es',
 'le',
 'e',
 'accide